# Motif price generator

This code parses TTC item and price tables and computes prices per motif

Copy SLPP.py here for ease of use, source is on github


In [1]:
from io import StringIO

In [2]:
import re
import sys
from numbers import Number

import six

ERRORS = {
    'unexp_end_string': u'Unexpected end of string while parsing Lua string.',
    'unexp_end_table': u'Unexpected end of table while parsing Lua string.',
    'mfnumber_minus': u'Malformed number (no digits after initial minus).',
    'mfnumber_dec_point': u'Malformed number (no digits after decimal point).',
    'mfnumber_sci': u'Malformed number (bad scientific format).',
}

def sequential(lst):
    length = len(lst)
    if length == 0 or lst[0] != 0:
        return False
    for i in range(length):
        if i + 1 < length:
            if lst[i] + 1 != lst[i+1]:
                return False
    return True


class ParseError(Exception):
    pass


class SLPP(object):

    def __init__(self):
        self.text = ''
        self.ch = ''
        self.at = 0
        self.len = 0
        self.depth = 0
        self.space = re.compile('\s', re.M)
        self.alnum = re.compile('\w', re.M)
        self.newline = '\n'
        self.tab = '\t'

    def decode(self, text):
        if not text or not isinstance(text, six.string_types):
            return
        # FIXME: only short comments removed
        reg = re.compile('--.*$', re.M)
        text = reg.sub('', text, 0)
        self.text = text
        self.at, self.ch, self.depth = 0, '', 0
        self.len = len(text)
        self.next_chr()
        result = self.value()
        return result

    def encode(self, obj):
        self.depth = 0
        return self.__encode(obj)

    def __encode(self, obj):
        s = ''
        tab = self.tab
        newline = self.newline

        if isinstance(obj, str):
            s += '"%s"' % obj.replace(r'"', r'\"')
        elif six.PY2 and isinstance(obj, unicode):
            s += '"%s"' % obj.encode('utf-8').replace(r'"', r'\"')
        elif six.PY3 and isinstance(obj, bytes):
            s += '"{}"'.format(''.join(r'\x{:02x}'.format(c) for c in obj))
        elif isinstance(obj, bool):
            s += str(obj).lower()
        elif obj is None:
            s += 'nil'
        elif isinstance(obj, Number):
            s += str(obj)
        elif isinstance(obj, (list, tuple, dict)):
            self.depth += 1
            if len(obj) == 0 or (not isinstance(obj, dict) and len([
                    x for x in obj
                    if isinstance(x, Number) or (isinstance(x, six.string_types) and len(x) < 10)
               ]) == len(obj)):
                newline = tab = ''
            dp = tab * self.depth
            s += "%s{%s" % (tab * (self.depth - 2), newline)
            if isinstance(obj, dict):
                key = '[%s]' if all(isinstance(k, (int, long)) for k in obj.keys()) else '%s'
                contents = [dp + (key + ' = %s') % (k, self.__encode(v)) for k, v in obj.items()]
                s += (',%s' % newline).join(contents)
            else:
                s += (',%s' % newline).join(
                    [dp + self.__encode(el) for el in obj])
            self.depth -= 1
            s += "%s%s}" % (newline, tab * self.depth)
        return s

    def white(self):
        while self.ch:
            if self.space.match(self.ch):
                self.next_chr()
            else:
                break

    def next_chr(self):
        if self.at >= self.len:
            self.ch = None
            return None
        self.ch = self.text[self.at]
        self.at += 1
        return True

    def value(self):
        self.white()
        if not self.ch:
            return
        if self.ch == '{':
            return self.object()
        if self.ch == "[":
            self.next_chr()
        if self.ch in ['"',  "'",  '[']:
            return self.string(self.ch)
        if self.ch.isdigit() or self.ch == '-':
            return self.number()
        return self.word()

    def string(self, end=None):
        s = ''
        start = self.ch
        if end == '[':
            end = ']'
        if start in ['"',  "'",  '[']:
            while self.next_chr():
                if self.ch == end:
                    self.next_chr()
                    if start != "[" or self.ch == ']':
                        return s
                if self.ch == '\\' and start == end:
                    self.next_chr()
                    if self.ch != end:
                        s += '\\'
                s += self.ch
        raise ParseError(ERRORS['unexp_end_string'])

    def object(self):
        o = {}
        k = None
        idx = 0
        numeric_keys = False
        self.depth += 1
        self.next_chr()
        self.white()
        if self.ch and self.ch == '}':
            self.depth -= 1
            self.next_chr()
            return o  # Exit here
        else:
            while self.ch:
                self.white()
                if self.ch == '{':
                    o[idx] = self.object()
                    idx += 1
                    continue
                elif self.ch == '}':
                    self.depth -= 1
                    self.next_chr()
                    if k is not None:
                        o[idx] = k
                    if len([key for key in o if isinstance(key, six.string_types + (float,  bool, tuple))]) == 0:
                        so = sorted([key for key in o])
                        if sequential(so):
                            ar = []
                            for key in o:
                                ar.insert(key, o[key])
                            o = ar
                    return o  # or here
                else:
                    if self.ch == ',':
                        self.next_chr()
                        continue
                    else:
                        k = self.value()
                        if self.ch == ']':
                            self.next_chr()
                    self.white()
                    ch = self.ch
                    if ch in ('=', ','):
                        self.next_chr()
                        self.white()
                        if ch == '=':
                            o[k] = self.value()
                        else:
                            o[idx] = k
                        idx += 1
                        k = None
        raise ParseError(ERRORS['unexp_end_table'])  # Bad exit here

    words = {'true': True, 'false': False, 'nil': None}
    def word(self):
        s = ''
        if self.ch != '\n':
            s = self.ch
        self.next_chr()
        while self.ch is not None and self.alnum.match(self.ch) and s not in self.words:
            s += self.ch
            self.next_chr()
        return self.words.get(s, s)

    def number(self):
        def next_digit(err):
            n = self.ch
            self.next_chr()
            if not self.ch or not self.ch.isdigit():
                raise ParseError(err)
            return n
        n = ''
        try:
            if self.ch == '-':
                n += next_digit(ERRORS['mfnumber_minus'])
            n += self.digit()
            if n == '0' and self.ch in ['x', 'X']:
                n += self.ch
                self.next_chr()
                n += self.hex()
            else:
                if self.ch and self.ch == '.':
                    n += next_digit(ERRORS['mfnumber_dec_point'])
                    n += self.digit()
                if self.ch and self.ch in ['e', 'E']:
                    n += self.ch
                    self.next_chr()
                    if not self.ch or self.ch not in ('+', '-'):
                        raise ParseError(ERRORS['mfnumber_sci'])
                    n += next_digit(ERRORS['mfnumber_sci'])
                    n += self.digit()
        except ParseError:
            t, e = sys.exc_info()[:2]
            print(e)
            return 0
        try:
            return int(n, 0)
        except:
            pass
        return float(n)

    def digit(self):
        n = ''
        while self.ch and self.ch.isdigit():
            n += self.ch
            self.next_chr()
        return n

    def hex(self):
        n = ''
        while self.ch and (self.ch in 'ABCDEFabcdef' or self.ch.isdigit()):
            n += self.ch
            self.next_chr()
        return n


slpp = SLPP()

__all__ = ['slpp']

Load TTC Files. They contain simple functions and we want just the data structure, so split them on "variable-name=" and keep the stuff on the right, which is the actual Lua table

In [3]:
item_lookup = open(r"C:\Users\jtern\Documents\Elder Scrolls Online\live\AddOns\TamrielTradeCentre\ItemLookUpTable_EN.lua",'r').read()

In [4]:
item_prices = open(r"C:\Users\jtern\Documents\Elder Scrolls Online\live\AddOns\TamrielTradeCentre\PriceTable.lua",'r').read()

In [5]:
prices = slpp.decode(item_prices.split('self.PriceTable=')[-1])

In [6]:
items = slpp.decode(item_lookup.split('self.ItemLookUpTable=')[-1])

In [7]:
# find all items whose name starts with "crafting motif"
motif_keys = [key for key in items.keys() if key.startswith('crafting motif')]

In [8]:
# show an example record. The Lua nested table structure looks like {item_id : {level : {traits : {'Avg' : avg...}}}}
# so we'll need to unnest it
prices['Data'][6444]

{2: {1: {-1: {'Avg': 639.44,
    'Max': 90000,
    'Min': 1,
    'EntryCount': 951,
    'AmountCount': 974,
    'SuggestedPrice': 80.17}}}}

In [9]:
# demonstrate the unnesting process
qual = list(prices['Data'][6444].keys())[0]
level_dict=prices['Data'][6444][qual]
level = list(level_dict.keys())[0]
traits_dict = level_dict[level]
traits = list(traits_dict.keys())[0]
prices_dict = traits_dict[traits]
print(qual,level, traits, prices_dict)

2 1 -1 {'Avg': 639.44, 'Max': 90000, 'Min': 1, 'EntryCount': 951, 'AmountCount': 974, 'SuggestedPrice': 80.17}


In [10]:
#Iterate over all the motifs and build a new list with prices and names. If prices are not found, leave a blank entry
data = []
for motif_key in motif_keys:
    item = items[motif_key]
    entry = {'motif' : motif_key}
    entry['item_id'] = list(item.items())[0][1]
    try:
        price = prices['Data'][entry['item_id']]
    except KeyError:
        data.append(entry)
        continue
    qual = list(price.keys())[0]
    level_dict=price[qual]
    level = list(level_dict.keys())[0]
    traits_dict = level_dict[level]
    traits = list(traits_dict.keys())[0]
    prices_dict = traits_dict[traits]
    entry = {**entry,**prices_dict}
    data.append(entry)

Import Pandas for data manipulation and make a data frame out of our new data

In [11]:
import pandas as pd
#set some pandas display options
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_rows', 1000)

In [12]:
motif_df = pd.DataFrame(data)
motif_df.head()

,motif,item_id,Avg,Max,Min,EntryCount,AmountCount,SuggestedPrice
0,crafting motif 16: glass legs,6328,"18,340.37","300,000.00","5,000.00",79.00,79.00,"8,012.48"
1,crafting motif 17: xivkyn belts,6329,"12,909.30","149,999.00","1,000.00",114.00,116.00,"3,859.89"
2,crafting motif 15: dwemer bows,6340,"6,595.83","225,000.00",700.00,150.00,150.00,"1,146.89"
3,crafting motif 21: ancient orc boots,6349,"13,675.36","30,000.00","5,000.00",25.00,25.00,"8,550.00"
4,crafting motif 19: mercenary axes,6352,"10,694.59","30,000.00",500.00,44.00,44.00,"5,229.83"


In [13]:
#extract the motif number from the full name
motif_df['motif_num'] = motif_df['motif'].str.extract("(\d+):",expand=True)

In [14]:
#extract the armor/weapon slot as the last word in the motif name
motif_df['motif_slot'] = motif_df['motif'].str.rsplit(' ',1,expand=True)[1]

In [15]:
# take everything between : and the last word as the motif name
motif_df['motif_name'] = motif_df['motif'].str.rsplit(' ',1,expand=True)[0].str.split(':',expand=True)[1].str.lstrip()

In [16]:
#make a numeric motif number for ordering instead of string, which would give 1,10,11,etc.
#calculate the minimum of Avg and SuggestedPrice as the new calculcated cost
motif_df['motif_sort_num'] = pd.to_numeric(motif_df['motif_num'])
motif_df['calc_cost'] = motif_df[['Avg','SuggestedPrice']].min(axis=1)

In [45]:
# if the motif slot is 'style' then this is the price for the whole book. Fill any missing values with 1m gold
motif_df['Style_Cost']=motif_df.loc[motif_df['motif_slot']=='style']['calc_cost']
# set soul shriven to 0
motif_df.loc[motif_df.motif_name.str.startswith('soul'),'Style_Cost']=0

In [46]:
# rank the page costs by motif in ascending order
motif_df['rank'] = motif_df.sort_values(['motif_sort_num','calc_cost'],ascending=[True,False]).groupby('motif_num')['calc_cost'].rank("dense",ascending=False)

In [47]:
motif_df.sort_values(['motif_sort_num','rank']).head(50)

,motif,item_id,Avg,Max,Min,EntryCount,AmountCount,SuggestedPrice,motif_num,motif_slot,motif_name,motif_sort_num,calc_cost,Style_Cost,rank
60,crafting motif 1: high elf style,6533,467.17,"30,000.00",1.00,"1,097.00","1,150.00",72.69,1,style,high elf,1,72.69,72.69,1.00
54,crafting motif 2: dark elf style,6511,417.17,"25,000.00",1.00,"1,073.00","1,091.00",78.75,2,style,dark elf,2,78.75,78.75,1.00
73,crafting motif 3: wood elf style,6580,499.52,"45,000.00",1.00,902.00,929.00,75.25,3,style,wood elf,3,75.25,75.25,1.00
59,crafting motif 4: nord style,6525,682.61,"50,000.00",1.00,899.00,919.00,76.53,4,style,nord,4,76.53,76.53,1.00
61,crafting motif 5: breton style,6534,391.42,"15,000.00",1.00,965.00,993.00,72.54,5,style,breton,5,72.54,72.54,1.00
47,crafting motif 6: redguard style,6500,590.68,"50,000.00",1.00,947.00,972.00,78.06,6,style,redguard,6,78.06,78.06,1.00
72,crafting motif 7: khajiit style,6573,385.84,"50,000.00",1.00,"1,145.00","1,182.00",69.56,7,style,khajiit,7,69.56,69.56,1.00
32,crafting motif 8: orc style,6444,639.44,"90,000.00",1.00,951.00,974.00,80.17,8,style,orc,8,80.17,80.17,1.00
82,crafting motif 9: argonian style,6612,459.91,"10,000.00",1.00,952.00,973.00,76.22,9,style,argonian,9,76.22,76.22,1.00
81,crafting motif 10: imperial style,6610,"130,976.70","274,089.00","5,000.00",446.00,446.00,"95,588.81",10,style,imperial,10,"95,588.81","95,588.81",1.00


In [48]:
# we can now do analytics on the costs. show what percentage of slots are the lowest cost, for example.
motif_df.loc[(motif_df['motif_slot']!='style') & (motif_df['rank']==15)]['motif_slot'].value_counts(normalize=True)

belts     0.30
axes      0.22
gloves    0.13
daggers   0.13
maces     0.13
shields   0.04
boots     0.04
Name: motif_slot, dtype: float64

In [49]:
# sum up the total motif cost (calc_cost)
by_page_df = motif_df.loc[motif_df['motif_slot'] != 'style'].groupby(['motif_sort_num','motif_name'])['calc_cost'].sum()

In [50]:
# create a new data frame with just prices for full styles
by_style_df = motif_df.loc[motif_df['motif_slot']=='style']

In [51]:
by_style_df.head()

,motif,item_id,Avg,Max,Min,EntryCount,AmountCount,SuggestedPrice,motif_num,motif_slot,motif_name,motif_sort_num,calc_cost,Style_Cost,rank
6,crafting motif 12: barbaric style,6359,"1,838.94","100,000.00",1.00,957.00,984.00,328.07,12,style,barbaric,12,328.07,328.07,1.00
7,crafting motif 14: daedric style,6362,"3,969.26","100,000.00",10.00,434.00,443.00,602.99,14,style,daedric,14,602.99,602.99,1.00
26,crafting motif 13: primal style,6431,"2,262.42","300,000.00",1.00,"1,151.00","1,161.00",311.67,13,style,primal,13,311.67,311.67,1.00
32,crafting motif 8: orc style,6444,639.44,"90,000.00",1.00,951.00,974.00,80.17,8,style,orc,8,80.17,80.17,1.00
47,crafting motif 6: redguard style,6500,590.68,"50,000.00",1.00,947.00,972.00,78.06,6,style,redguard,6,78.06,78.06,1.00


In [52]:
# create a new cost df by joining by_page and by_style on motif_sort_num. Use outer join because some styles don't have pages and vice-versa
cost_df = by_page_df.reset_index().merge(by_style_df[['motif_sort_num','Style_Cost','motif_name']],on='motif_sort_num',how='outer')

In [53]:
cost_df.head()

,motif_sort_num,motif_name_x,calc_cost,Style_Cost,motif_name_y
0,15,dwemer,"16,952.41",nan,dwemer
1,16,glass,"93,474.66","159,959.50",glass
2,17,xivkyn,"70,498.25","1,290,000.00",xivkyn
3,18,akaviri,"246,714.00",nan,akaviri
4,19,mercenary,"186,624.62",nan,mercenary


In [54]:
# compute the lesser of the summed page cost and style cost
cost_df['min_cost'] = cost_df[['calc_cost','Style_Cost']].min(axis=1)

In [55]:
# add a cumulative cost of buying the cheapest motifs in order
cost_df['cumulative_cost'] = cost_df.sort_values('min_cost')['min_cost'].cumsum()

In [56]:
# take the two motif names from the outer join and make a single non-null motif name
cost_df['motif_name'] = cost_df.apply(lambda r : r['motif_name_x'] if pd.notnull(r['motif_name_x']) else r['motif_name_y'],axis=1)

In [57]:
#drop the extra name columns and make names friendlier
new_cost_df = cost_df.drop(columns=['motif_name_x','motif_name_y']).rename(columns={'calc_cost':'ByPage_Cost', 'motif_sort_num':'motif_number'}).sort_values('min_cost').reset_index(drop=True)

In [58]:
# hide prices where the book is more expensive than the by_page cost
import numpy as np
new_cost_df['Style_Cost'] = new_cost_df.apply(lambda r : np.nan if (pd.notna(r['ByPage_Cost']) and (r['ByPage_Cost']<r['Style_Cost'])) else r['Style_Cost'],axis=1)

In [59]:
new_cost_df.fillna('')

,motif_number,ByPage_Cost,Style_Cost,min_cost,cumulative_cost,motif_name
0,29,,0.00,0.00,0.00,soul shriven
1,7,,69.56,69.56,69.56,khajiit
2,5,,72.54,72.54,142.10,breton
3,1,,72.69,72.69,214.79,high elf
4,3,,75.25,75.25,290.04,wood elf
5,9,,76.22,76.22,366.26,argonian
6,4,,76.53,76.53,442.79,nord
7,6,,78.06,78.06,520.85,redguard
8,2,,78.75,78.75,599.60,dark elf
9,8,,80.17,80.17,679.77,orc


In [60]:
# Format the numeric columns nicely
for col in ['cumulative_cost','min_cost','ByPage_Cost','Style_Cost']:
    new_cost_df[col] = new_cost_df[col].apply(lambda x : "{:,.2f}".format(x) if pd.notna(x) else '')
new_cost_df.reset_index(inplace=True)
new_cost_df.head()


,index,motif_number,ByPage_Cost,Style_Cost,min_cost,cumulative_cost,motif_name
0,0,29,,0.00,0.00,0.00,soul shriven
1,1,7,,69.56,69.56,69.56,khajiit
2,2,5,,72.54,72.54,142.10,breton
3,3,1,,72.69,72.69,214.79,high elf
4,4,3,,75.25,75.25,290.04,wood elf


In [63]:
new_cost_df.fillna('').head(50)

,index,motif_number,ByPage_Cost,Style_Cost,min_cost,cumulative_cost,motif_name
0,0,29,,0.00,0.00,0.00,soul shriven
1,1,2,,76.53,76.53,76.53,dark elf
2,2,5,,77.78,77.78,154.31,breton
3,3,3,,80.49,80.49,234.80,wood elf
4,4,9,,80.98,80.98,315.78,argonian
5,5,7,,81.42,81.42,397.20,khajiit
6,6,8,,85.65,85.65,482.85,orc
7,7,6,,88.20,88.20,571.05,redguard
8,8,4,,89.92,89.92,660.97,nord
9,9,1,,102.96,102.96,763.93,high elf


In [33]:
# Use StringIO to write the data frame in a reddit-friendly manner (pipe delimited)
from io import StringIO

In [34]:
buf = StringIO()
new_cost_df.to_csv(buf,index=False,sep='|')

In [35]:
# reset the buffer to start, and read the buffer into a string
buf.seek(0)

0

In [36]:
print(buf.readline().strip())
print(':--|--:|--:|--:|--:|--:|:--')
for line in buf.readlines():
    print(line.strip())

index|motif_number|ByPage_Cost|Style_Cost|min_cost|cumulative_cost|motif_name
:--|--:|--:|--:|--:|--:|:--
0|29||0.00|0.00|0.00|soul shriven
1|8||59.41|59.41|59.41|orc
2|3||60.68|60.68|120.09|wood elf
3|1||64.10|64.10|184.19|high elf
4|4||64.18|64.18|248.37|nord
5|5||66.12|66.12|314.49|breton
6|9||66.15|66.15|380.64|argonian
7|6||66.75|66.75|447.39|redguard
8|7||70.75|70.75|518.14|khajiit
9|2||71.44|71.44|589.58|dark elf
10|13||207.90|207.90|797.48|primal
11|12||268.74|268.74|1,066.22|barbaric
12|11||278.51|278.51|1,344.73|ancient elf
13|14||757.92|757.92|2,102.65|daedric
14|31|12,254.49||12,254.49|14,357.14|skinchanger
15|15|15,751.08||15,751.08|30,108.22|dwemer
16|81|16,359.24||16,359.24|46,467.46|new moon priest
17|28|24,558.32||24,558.32|71,025.78|ra gada
18|84|28,070.95||28,070.95|99,096.73|blackreach vanguard
19|57|44,454.44||44,454.44|143,551.17|ebonshadow
20|33|47,440.46||47,440.46|190,991.63|thieves guild
21|48|47,914.28||47,914.28|238,905.91|ashlander
22|63|49,712.14||49,712.1

In [77]:
print(s)

NameError: name 's' is not defined

In [503]:
new_items = []
for _item,_item_dict in items.items():
    _item_entry = {'name' : _item}
    if isinstance(_item_dict, dict) :
        _item_entry['type'] = list(_item_dict.keys())[0]
        _item_entry['ttc_id'] = _item_dict[list(_item_dict.keys())[0]]
    else:
        _item_entry['type'] = None
        _item_entry['ttc_id'] = _item_dict[0]
    #_prices_sub = prices['Data'][item_entry['ttc_id']]
    new_items.append(_item_entry)
        
    

In [501]:
prices['Data'][183]

{2: {1: {-1: {'Avg': 1034.25,
    'Max': 2500,
    'Min': 180,
    'EntryCount': 8,
    'AmountCount': 8}}}}

In [506]:
items_new_df = pd.DataFrame(new_items)

In [525]:
items_new_df.sort_values('type').groupby('type').first()

,name,ttc_id
type,,
1.00,roast pig,1980
2.00,congealed melon jelly,7001
3.00,cold jerall view inn carrot cake,6822
4.00,melon-baked parmesan pork,5187
5.00,garlic cod with potato crust,6437
6.00,late-summer corn slaw,6236
7.00,dawnstar sun's dusk chowder,5593
8.00,stale bread,14740
20.00,flat nut brown ale,11778


In [508]:
items_new_df['type'].unique()

array([4.500e+02, 3.000e+02, 2.100e+01, 1.250e+03, 9.500e+02, 2.500e+02,
             nan, 2.400e+01, 2.300e+01, 1.710e+02, 2.000e+00, 1.650e+03,
       1.020e+02, 1.700e+02, 1.000e+03, 2.600e+01, 6.000e+00, 1.000e+02,
       1.700e+03, 7.000e+00, 5.000e+00, 2.200e+01, 1.500e+02, 7.500e+02,
       4.000e+00, 2.300e+03, 1.750e+03, 2.000e+01, 1.040e+02, 1.900e+03,
       4.300e+01, 1.030e+02, 1.450e+03, 2.050e+03, 2.400e+03, 4.040e+02,
       1.500e+03, 9.000e+02, 1.000e+00, 2.500e+01, 4.000e+02, 1.510e+02,
       3.000e+00, 1.600e+03, 4.010e+02, 2.350e+03, 1.550e+03, 2.000e+03,
       4.070e+02, 2.450e+03, 1.950e+03, 4.200e+01, 1.800e+03, 4.700e+01,
       5.000e+02, 4.000e+01, 8.500e+02, 4.100e+01, 4.500e+01, 2.100e+03,
       1.850e+03, 8.000e+02, 4.020e+02, 4.400e+01, 4.060e+02, 4.600e+01,
       2.150e+03, 4.800e+01, 2.700e+01, 6.100e+01, 8.000e+00, 6.000e+01,
       2.550e+03, 2.600e+03, 1.520e+02, 2.650e+03, 1.400e+03, 8.000e+01,
       1.460e+03, 1.660e+03, 1.760e+03, 1.465e+03, 

In [64]:
new_cost_df

,index,motif_number,ByPage_Cost,Style_Cost,min_cost,cumulative_cost,motif_name
0,0,29,,0.00,0.00,0.00,soul shriven
1,1,2,,76.53,76.53,76.53,dark elf
2,2,5,,77.78,77.78,154.31,breton
3,3,3,,80.49,80.49,234.80,wood elf
4,4,9,,80.98,80.98,315.78,argonian
5,5,7,,81.42,81.42,397.20,khajiit
6,6,8,,85.65,85.65,482.85,orc
7,7,6,,88.20,88.20,571.05,redguard
8,8,4,,89.92,89.92,660.97,nord
9,9,1,,102.96,102.96,763.93,high elf


In [32]:
by_style_df

,motif,item_id,Avg,Max,Min,EntryCount,AmountCount,SuggestedPrice,motif_num,motif_slot,motif_name,motif_sort_num,calc_cost,Style_Cost,rank
6,crafting motif 12: barbaric style,6359,367.33,"100,000.00",1.00,"1,351.00","1,377.00",330.60,12,style,barbaric,12,330.60,330.60,1.00
7,crafting motif 14: daedric style,6362,"1,164.49","35,000.00",99.00,752.00,758.00,"1,048.04",14,style,daedric,14,"1,048.04","1,048.04",1.00
26,crafting motif 13: primal style,6431,384.03,"20,000.00",50.00,"1,713.00","1,732.00",345.62,13,style,primal,13,345.62,345.62,1.00
32,crafting motif 8: orc style,6444,103.74,"30,000.00",1.00,"1,405.00","1,444.00",93.36,8,style,orc,8,93.36,93.36,1.00
47,crafting motif 6: redguard style,6500,102.95,"50,000.00",5.00,"1,355.00","1,387.00",92.65,6,style,redguard,6,92.65,92.65,1.00
49,crafting motif 11: ancient elf style,6503,419.71,"90,000.00",1.00,"1,235.00","1,245.00",377.74,11,style,ancient elf,11,377.74,377.74,1.00
54,crafting motif 2: dark elf style,6511,98.89,"15,000.00",2.13,"1,392.00","1,473.00",89.00,2,style,dark elf,2,89.00,89.00,1.00
59,crafting motif 4: nord style,6525,101.31,"800,000.00",1.00,"1,498.00","1,564.00",91.18,4,style,nord,4,91.18,91.18,1.00
60,crafting motif 1: high elf style,6533,94.58,"13,000.00",1.00,"1,533.00","1,578.00",85.12,1,style,high elf,1,85.12,85.12,1.00
61,crafting motif 5: breton style,6534,93.95,"20,000.00",3.33,"1,502.00","1,568.00",84.56,5,style,breton,5,84.56,84.56,1.00


In [33]:
by_page_df

motif_sort_num  motif_name          
15               dwemer                 22,371.85
16               glass                  59,407.39
17               xivkyn                 74,860.60
18               akaviri               196,645.56
19               mercenary              70,546.80
20               yokudan               113,625.88
21               ancient orc           163,903.78
22               trinimac              126,611.02
23               malacath               85,298.66
24               outlaw                 95,180.07
25               aldmeri dominion       53,152.36
26               daggerfall covenant    61,110.74
27               ebonheart pact         71,938.35
28               ra gada                18,010.96
30               morag tong             91,644.96
31               skinchanger            25,098.53
32               abah's watch           81,424.26
33               thieves guild          17,275.89
34               assassins league       98,018.47
35           

In [35]:
by_page_df

motif_sort_num  motif_name          
15               dwemer                 24,857.62
16               glass                  66,008.20
17               xivkyn                 83,178.44
18               akaviri               218,495.07
19               mercenary              78,385.33
20               yokudan               126,250.97
21               ancient orc           182,115.31
22               trinimac              140,678.92
23               malacath               94,776.30
24               outlaw                105,755.64
25               aldmeri dominion       59,058.15
26               daggerfall covenant    67,900.82
27               ebonheart pact         79,931.51
28               ra gada                20,012.18
30               morag tong            101,827.75
31               skinchanger            27,887.24
32               abah's watch           90,471.41
33               thieves guild          19,195.42
34               assassins league      108,909.39
35           

In [32]:
from io import StringIO
buf = StringIO()

In [33]:
motif_df.loc[motif_df.motif_name==' moongrave fane'][['motif','Min','Avg','Max','SuggestedPrice','AmountCount']].to_csv(buf,sep='|',index=False)

In [38]:
motif_df.motif_name.unique()

array([' glass', ' xivkyn', ' dwemer', ' ancient orc', ' mercenary',
       ' akaviri', ' barbaric', ' daedric', ' primal', ' orc',
       ' redguard', ' ancient elf', ' dark elf', ' nord', ' high elf',
       ' breton', ' khajiit', ' wood elf', ' imperial', ' argonian',
       ' ebonheart pact', ' trinimac', ' daggerfall covenant', ' outlaw',
       ' aldmeri dominion', ' malacath', ' thieves guild',
       ' soul shriven', ' assassins league', " abah's watch", ' minotaur',
       " dro-m'athra", ' order hour', ' dark brotherhood', ' celestial',
       ' yokudan', ' draugr', ' skinchanger', ' hollowjack',
       ' silken ring', ' mazzatun', ' ra gada', ' ebony', ' morag tong',
       ' militant ordinator', ' buoyant armiger', ' ashlander',
       ' telvanni', ' hlaalu', ' redoran', ' ebonshadow', ' dreadhorn',
       ' apostle', ' bloodforge', ' worm cult', ' psijic', ' sapiarch',
       ' pyandonean', ' fang lair', ' scalecaller', ' welkynar',
       ' dremora', ' dead-water', ' silv

In [37]:
buf.seek(0)

0

In [38]:
print(buf.read())

index|motif_number|ByPage_Cost|Style_Cost|min_cost|cumulative_cost|motif_name
0|29||0.00|0.00|0.00|soul shriven
1|7||42.23|42.23|42.23|khajiit
2|8||45.83|45.83|88.06|orc
3|2||46.27|46.27|134.33|dark elf
4|9||47.66|47.66|181.99|argonian
5|4||48.13|48.13|230.12|nord
6|5||52.14|52.14|282.26|breton
7|1||52.17|52.17|334.43|high elf
8|3||52.19|52.19|386.62|wood elf
9|6||54.23|54.23|440.85|redguard
10|13||174.62|174.62|615.47|primal
11|12||194.19|194.19|809.66|barbaric
12|11||203.34|203.34|1,013.00|ancient elf
13|14||423.44|423.44|1,436.44|daedric
14|15|11,879.58||11,879.58|13,316.02|dwemer
15|28|13,995.88||13,995.88|27,311.90|ra gada
16|81|22,439.10||22,439.10|49,751.00|new moon priest
17|84|24,681.21||24,681.21|74,432.21|blackreach vanguard
18|62|25,804.99||25,804.99|100,237.20|sapiarch
19|33|34,781.04||34,781.04|135,018.24|thieves guild
20|63|51,514.26||51,514.26|186,532.50|dremora
21|25|55,006.50||55,006.50|241,539.00|aldmeri dominion
22|88|58,279.55||58,279.55|299,818.55|ancestral orc
23

In [ ]:
for col in ['Min','Avg','Max','SuggestedPrice']:
    new_cost_df[col] = new_cost_df[col].apply(lambda x : "{:,.2f}".format(x))


In [39]:
buf.seek(0)
print(buf.readline().strip())
print(':--|:--|--:|--:|--:|:--')
for line in buf.readlines():
    print(line.strip())

index|motif_number|ByPage_Cost|Style_Cost|min_cost|cumulative_cost|motif_name
:--|:--|--:|--:|--:|:--
0|29||0.00|0.00|0.00|soul shriven
1|7||42.23|42.23|42.23|khajiit
2|8||45.83|45.83|88.06|orc
3|2||46.27|46.27|134.33|dark elf
4|9||47.66|47.66|181.99|argonian
5|4||48.13|48.13|230.12|nord
6|5||52.14|52.14|282.26|breton
7|1||52.17|52.17|334.43|high elf
8|3||52.19|52.19|386.62|wood elf
9|6||54.23|54.23|440.85|redguard
10|13||174.62|174.62|615.47|primal
11|12||194.19|194.19|809.66|barbaric
12|11||203.34|203.34|1,013.00|ancient elf
13|14||423.44|423.44|1,436.44|daedric
14|15|11,879.58||11,879.58|13,316.02|dwemer
15|28|13,995.88||13,995.88|27,311.90|ra gada
16|81|22,439.10||22,439.10|49,751.00|new moon priest
17|84|24,681.21||24,681.21|74,432.21|blackreach vanguard
18|62|25,804.99||25,804.99|100,237.20|sapiarch
19|33|34,781.04||34,781.04|135,018.24|thieves guild
20|63|51,514.26||51,514.26|186,532.50|dremora
21|25|55,006.50||55,006.50|241,539.00|aldmeri dominion
22|88|58,279.55||58,279.55|299

In [36]:
buf.readline()

'|motif_number|ByPage_Cost|Style_Cost|min_cost|cumulative_cost|motif_name\r\n'

In [61]:
motif_df.loc[motif_df.motif_name.str.startswith(' soul')]

,motif,item_id,Avg,Max,Min,EntryCount,AmountCount,SuggestedPrice,motif_num,motif_slot,motif_name,motif_sort_num,calc_cost,Style_Cost,rank
186,crafting motif 29: soul shriven style,7949,nan,nan,nan,nan,nan,nan,29,style,soul shriven,29,nan,"1,000,000.00",nan


In [84]:
[key for key in items.keys() if 'legion' in key]

['mark of the legion',
 'boots of the seventh legion',
 'fifth legion porter',
 'recipe: fifth legion porter',
 "seventh legion's necklace",
 "seventh legion's axe",
 "seventh legion's greaves",
 "seventh legion's conjured oath-band",
 "seventh legion's girdle",
 "seventh legion's shield",
 'seventh legion gauntlets',
 "seventh legion's maul",
 "seventh legion's helm",
 "seventh legion's sabatons",
 "seventh legion's cuirass",
 "seventh legion's restoration staff",
 "seventh legion's ring",
 "seventh legion's ice staff",
 "seventh legion's the wolfguard",
 "seventh legion's lightning staff",
 "seventh legion's dagger",
 "seventh legion's battle axe",
 "seventh legion's sword",
 "seventh legion's bow",
 "seventh legion's pauldron",
 "seventh legion's mace",
 "seventh legion's lava-charred greaves",
 "seventh legion's gauntlets",
 "seventh legion's ayleid breastplate",
 "seventh legion's greatsword",
 "seventh legion's inferno staff",
 "ex-legionairy's breastplate of soulshine",
 'might 

In [34]:
last_prices = """
rank|motif_number|By Page|By Book|Best|Cumulative|motif_name|Prior|Change
0|29||0.00|0.00|0.00|soul shriven|0|+0
1|8||68.35|68.35|68.35|orc|1|+0
2|2||70.92|70.92|139.27|dark elf|9|-7
3|5||74.15|74.15|213.42|breton|5|-2
4|4||74.55|74.55|287.97|nord|4|+0
5|3||74.64|74.64|362.61|wood elf|2|+3
6|6||76.18|76.18|438.79|redguard|7|-1
7|7||76.60|76.60|515.39|khajiit|8|-1
8|1||82.45|82.45|597.84|high elf|3|+5
9|9||83.90|83.90|681.74|argonian|6|+3
10|13||317.07|317.07|998.81|primal|10|+0
11|12||317.89|317.89|1,316.70|barbaric|11|+0
12|11||365.50|365.50|1,682.20|ancient elf|12|+0
13|14||848.12|848.12|2,530.32|daedric|13|+0
14|15|11,597.35||11,597.35|14,127.67|dwemer|15|-1
15|31|12,168.45||12,168.45|26,296.12|skinchanger|14|+1
16|84|16,562.19||16,562.19|42,858.31|blackreach vanguard|18|-2
17|81|17,264.28||17,264.28|60,122.59|new moon priest|16|+1
18|27|51,007.57|20,000.00|20,000.00|80,122.59|ebonheart pact|29|-11
19|28|24,257.56||24,257.56|104,380.15|ra gada|17|+2
20|33|27,173.35||27,173.35|131,553.50|thieves guild|20|+0
21|62|31,740.77||31,740.77|163,294.27|sapiarch|25|-4
22|57|32,602.73||32,602.73|195,897.00|ebonshadow|19|+3
23|48|33,738.98||33,738.98|229,635.98|ashlander|21|+2
24|85|40,620.98||40,620.98|270,256.96|greymoor|23|+1
25|25|44,766.62||44,766.62|315,023.58|aldmeri dominion|26|-1
26|26|56,361.38||56,361.38|371,384.96|daggerfall covenant|31|-5
27|42|57,421.00||57,421.00|428,805.96|hollowjack|24|+3
28|73|60,921.69||60,921.69|489,727.65|anequina|33|-5
29|17|69,586.23||69,586.23|559,313.88|xivkyn|42|-13
30|38|71,811.76||71,811.76|631,125.64|draugr|43|-13
31|44|74,046.40||74,046.40|705,172.04|silken ring|45|-14
32|56|74,338.57||74,338.57|779,510.61|apostle|32|+0
33|16|74,674.53||74,674.53|854,185.14|glass|47|-14
34|55|78,669.70||78,669.70|932,854.84|dreadhorn|41|-7
35|87|79,558.20||79,558.20|1,012,413.04|ancestral nord|28|+7
36|52|80,993.47||80,993.47|1,093,406.51|redoran|37|-1
37|61|83,239.55||83,239.55|1,176,646.06|psijic|36|+1
38|88|85,023.63||85,023.63|1,261,669.69|ancestral orc|27|+11
39|10||86,004.13|86,004.13|1,347,673.82|imperial|30|+9
40|30|87,286.24||87,286.24|1,434,960.06|morag tong|35|+5
41|36|87,510.72||87,510.72|1,522,470.78|dark brotherhood|48|-7
42|50|87,785.71||87,785.71|1,610,256.49|telvanni|40|+2
43|59|88,006.23||88,006.23|1,698,262.72|scalecaller|44|-1
44|75|88,405.69||88,405.69|1,786,668.41|sunspire|38|+6
45|51|89,713.87||89,713.87|1,876,382.28|hlaalu|39|+6
46|63|97,068.77||97,068.77|1,973,451.05|dremora|22|+24
47|34|105,051.14||105,051.14|2,078,502.19|assassins league|55|-8
48|54|107,860.12||107,860.12|2,186,362.31|bloodforge|52|-4
49|32|108,591.27||108,591.27|2,294,953.58|abah's watch|51|-2
50|96|117,643.17||117,643.17|2,412,596.75|arkthzand armory|46|+4
51|69|121,745.79||121,745.79|2,534,342.54|dead-water|54|-3
52|70|122,641.67||122,641.67|2,656,984.21|elder argonian|57|-5
53|20|131,485.37||131,485.37|2,788,469.58|yokudan|58|-5
54|41|135,561.88||135,561.88|2,924,031.46|celestial|53|+1
55|24|140,693.24||140,693.24|3,064,724.70|outlaw|59|-4
56|76|143,095.82||143,095.82|3,207,820.52|dragonguard|60|-4
57|67|143,842.64||143,842.64|3,351,663.16|welkynar|50|+7
58|23|154,648.25||154,648.25|3,506,311.41|malacath|63|-5
59|64|161,538.58||161,538.58|3,667,849.99|pyandonean|61|-2
60|65|162,289.61||162,289.61|3,830,139.60|huntsman|56|+4
61|19|178,617.32||178,617.32|4,008,756.92|mercenary|66|-5
62|74|186,360.06||186,360.06|4,195,116.98|pellitine|62|+0
63|21|197,406.69||197,406.69|4,392,523.67|ancient orc|64|-1
64|40|204,409.75||204,409.75|4,596,933.42|order hour|67|-3
65|37|482,378.50|222,284.70|222,284.70|4,819,218.12|ebony|49|+16
66|45|235,723.51||235,723.51|5,054,941.63|mazzatun|65|+1
67|18|256,154.98||256,154.98|5,311,096.61|akaviri|69|-2
68|22|263,667.18||263,667.18|5,574,763.79|trinimac|77|-9
69|89|266,190.69||266,190.69|5,840,954.48|ancestral high elf|34|+35
70|66|286,533.27||286,533.27|6,127,487.75|silver dawn|70|+0
71|60|291,985.23||291,985.23|6,419,472.98|worm cult|78|-7
72|58|314,631.80||314,631.80|6,734,104.78|fang lair|71|+1
73|47|333,402.39||333,402.39|7,067,507.17|buoyant armiger|74|-1
74|79|348,557.99||348,557.99|7,416,065.16|refabricated|73|+1
75|78|358,429.11||358,429.11|7,774,494.27|moongrave fane|75|+0
76|68|358,845.57||358,845.57|8,133,339.84|honor guard|72|+4
77|86|359,203.53||359,203.53|8,492,543.37|sea giant|68|+9
78|72|374,004.59||374,004.59|8,866,547.96|meridian|79|-1
79|71|453,074.34||453,074.34|9,319,622.30|coldsnap|81|-2
80|82|455,130.59||455,130.59|9,774,752.89|icereach coven|76|+4
81|80|512,236.40||512,236.40|10,286,989.29|shield of senchal|80|+1
82|77|564,553.17||564,553.17|10,851,542.46|stags of z'en|84|-2
83|83|585,682.07||585,682.07|11,437,224.53|pyre watch|82|+1
84|91|714,541.25||714,541.25|12,151,765.78|hazardous alchemy|83|+1
85|39|722,435.08||722,435.08|12,874,200.86|minotaur|86|-1
86|90|832,246.34||832,246.34|13,706,447.20|thorn legion|85|+1
87|35|1,111,473.71||1,111,473.71|14,817,920.91|dro-m'athra|87|+0
88|49|1,613,840.03||1,613,840.03|16,431,760.94|militant ordinator|88|+0
89|94|1,818,569.17||1,818,569.17|18,250,330.11|ancestral reach|89|+0
"""

In [35]:
last_price_df = pd.read_csv(StringIO(last_prices),delimiter='|')

In [36]:
last_price_df

,rank,motif_number,By Page,By Book,Best,Cumulative,motif_name,Prior,Change
0,0,29,NaN,0.00,0.00,0.00,soul shriven,0,0
1,1,8,NaN,68.35,68.35,68.35,orc,1,0
2,2,2,NaN,70.92,70.92,139.27,dark elf,9,-7
3,3,5,NaN,74.15,74.15,213.42,breton,5,-2
4,4,4,NaN,74.55,74.55,287.97,nord,4,0
5,5,3,NaN,74.64,74.64,362.61,wood elf,2,3
6,6,6,NaN,76.18,76.18,438.79,redguard,7,-1
7,7,7,NaN,76.60,76.60,515.39,khajiit,8,-1
8,8,1,NaN,82.45,82.45,597.84,high elf,3,5
9,9,9,NaN,83.90,83.90,681.74,argonian,6,3


In [37]:
new_cost_df

,index,motif_number,ByPage_Cost,Style_Cost,min_cost,cumulative_cost,motif_name
0,0,29,,0.00,0.00,0.00,soul shriven
1,1,7,,69.56,69.56,69.56,khajiit
2,2,5,,72.54,72.54,142.10,breton
3,3,1,,72.69,72.69,214.79,high elf
4,4,3,,75.25,75.25,290.04,wood elf
5,5,9,,76.22,76.22,366.26,argonian
6,6,4,,76.53,76.53,442.79,nord
7,7,6,,78.06,78.06,520.85,redguard
8,8,2,,78.75,78.75,599.60,dark elf
9,9,8,,80.17,80.17,679.77,orc


In [61]:
diff_df = new_cost_df.merge(last_price_df[['rank','motif_name']],how='outer',on='motif_name')

In [62]:
diff_df

,index,motif_number,ByPage_Cost,Style_Cost,min_cost,cumulative_cost,motif_name,rank
0,0,29,,0.00,0.00,0.00,soul shriven,0.00
1,1,7,,69.56,69.56,69.56,khajiit,7.00
2,2,5,,72.54,72.54,142.10,breton,3.00
3,3,1,,72.69,72.69,214.79,high elf,8.00
4,4,3,,75.25,75.25,290.04,wood elf,5.00
5,5,9,,76.22,76.22,366.26,argonian,9.00
6,6,4,,76.53,76.53,442.79,nord,4.00
7,7,6,,78.06,78.06,520.85,redguard,6.00
8,8,2,,78.75,78.75,599.60,dark elf,2.00
9,9,8,,80.17,80.17,679.77,orc,1.00


In [63]:
diff_df['change']=diff_df.apply(lambda r : '{0:+d}'.format(r['index']-int(r['rank'])) if pd.notna(r['rank']) else np.NaN,axis=1)

In [64]:
diff_df.rename(columns={'rank':'Prior','index':'Rank'},inplace=True)

In [65]:
diff_df['Prior'] = diff_df['Prior'].apply(lambda v : str(int(v)) if pd.notna(v) else 'NEW')
diff_df['change'] = diff_df['change'].apply(lambda v : str(v) if pd.notna(v) else 'NEW')

In [66]:
diff_df

,Rank,motif_number,ByPage_Cost,Style_Cost,min_cost,cumulative_cost,motif_name,Prior,change
0,0,29,,0.00,0.00,0.00,soul shriven,0,+0
1,1,7,,69.56,69.56,69.56,khajiit,7,-6
2,2,5,,72.54,72.54,142.10,breton,3,-1
3,3,1,,72.69,72.69,214.79,high elf,8,-5
4,4,3,,75.25,75.25,290.04,wood elf,5,-1
5,5,9,,76.22,76.22,366.26,argonian,9,-4
6,6,4,,76.53,76.53,442.79,nord,4,+2
7,7,6,,78.06,78.06,520.85,redguard,6,+1
8,8,2,,78.75,78.75,599.60,dark elf,2,+6
9,9,8,,80.17,80.17,679.77,orc,1,+8


In [67]:
buf = StringIO()
colnames = ['Rank','Motif Num','By Page', 'By Book', 'Best', 'Cumulative','Motif','Prior','Change']
diff_df.to_csv(buf,index=False,sep='|',header=colnames)
buf.seek(0)
print(buf.readline().strip())
print(':--|:--|--:|--:|--:|:--|:--|:--|:--')
for line in buf.readlines():
    print(line.strip())

Rank|Motif Num|By Page|By Book|Best|Cumulative|Motif|Prior|Change
:--|:--|--:|--:|--:|:--|:--|:--|:--
0|29||0.00|0.00|0.00|soul shriven|0|+0
1|7||69.56|69.56|69.56|khajiit|7|-6
2|5||72.54|72.54|142.10|breton|3|-1
3|1||72.69|72.69|214.79|high elf|8|-5
4|3||75.25|75.25|290.04|wood elf|5|-1
5|9||76.22|76.22|366.26|argonian|9|-4
6|4||76.53|76.53|442.79|nord|4|+2
7|6||78.06|78.06|520.85|redguard|6|+1
8|2||78.75|78.75|599.60|dark elf|2|+6
9|8||80.17|80.17|679.77|orc|1|+8
10|13||311.67|311.67|991.44|primal|10|+0
11|12||328.07|328.07|1,319.51|barbaric|11|+0
12|11||428.79|428.79|1,748.30|ancient elf|12|+0
13|14||602.99|602.99|2,351.29|daedric|13|+0
14|15|16,952.41||16,952.41|19,303.70|dwemer|14|+0
15|81|18,733.22||18,733.22|38,036.92|new moon priest|17|-2
16|28|25,521.69||25,521.69|63,558.61|ra gada|19|-3
17|84|27,958.96||27,958.96|91,517.57|blackreach vanguard|16|+1
18|48|34,769.38||34,769.38|126,286.95|ashlander|23|-5
19|57|36,317.82||36,317.82|162,604.77|ebonshadow|22|-3
20|62|36,514.22||36,

In [51]:
motif_df.loc[motif_df['motif'].str.contains('honor guard')]

,motif,item_id,Avg,Max,Min,EntryCount,AmountCount,SuggestedPrice,motif_num,motif_slot,motif_name,motif_sort_num,calc_cost,Style_Cost,rank
783,crafting motif 68: honor guard maces,20066,"11,754.95","72,500.00","1,200.00",215.00,222.00,"8,419.10",68,maces,honor guard,68,"8,419.10",nan,14.00
786,crafting motif 68: honor guard belts,20083,"15,185.40","34,000.00","1,500.00",216.00,216.00,"11,091.20",68,belts,honor guard,68,"11,091.20",nan,9.00
788,crafting motif 68: honor guard swords,20098,"14,902.42","80,000.00","5,000.00",205.00,206.00,"10,102.96",68,swords,honor guard,68,"10,102.96",nan,11.00
789,crafting motif 68: honor guard legs,20108,"41,101.55","132,000.00","3,000.00",280.00,280.00,"29,180.10",68,legs,honor guard,68,"29,180.10",nan,4.00
795,crafting motif 68: honor guard helmets,20190,"57,947.39","175,000.00","20,000.00",310.00,310.00,"42,061.14",68,helmets,honor guard,68,"42,061.14",nan,3.00
796,crafting motif 68: honor guard daggers,20239,"15,055.41","120,000.00","5,000.00",228.00,228.00,"9,068.29",68,daggers,honor guard,68,"9,068.29",nan,13.00
797,crafting motif 68: honor guard bows,20241,"13,779.66","45,000.00","3,200.00",230.00,231.00,"9,675.14",68,bows,honor guard,68,"9,675.14",nan,12.00
799,crafting motif 68: honor guard chests,20264,"82,180.63","749,999.00","20,000.00",339.00,341.00,"58,631.74",68,chests,honor guard,68,"58,631.74",nan,2.00
800,crafting motif 68: honor guard boots,20271,"19,054.76","87,454.00","6,999.00",202.00,202.00,"12,466.43",68,boots,honor guard,68,"12,466.43",nan,8.00
801,crafting motif 68: honor guard gloves,20273,"24,803.19","270,000.00","4,999.00",226.00,226.00,"16,186.53",68,gloves,honor guard,68,"16,186.53",nan,6.00
